# Model Training


In [13]:
## MODEL TRAINING.py
# Basic Libraries
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
## Modelling Libraries:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
import warnings

### Import the CSV Data as Pandas Data Frame


In [3]:
df = pd.read_csv('data/stud.csv')

In [4]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


#### Setting up X and Y variables

In [5]:
x = df.drop(columns='math_score',axis=1)
y = df['math_score']

#### Create column transformer with 3 types of transformers

In [6]:
## Will seperate numerical and categorical features , for columnTransformer
num_features = x.select_dtypes(exclude='object').columns
cat_features = x.select_dtypes(include='object').columns

## # Preprocessor Logic:
##### OneHotEncoder → converts categorical features to binary columns
##### StandardScaler → scales numeric features to zero mean & unit variance
##### ColumnTransformer(parent func, assigns the baby funcs stdscalar and onhot..theoir required inputs,) applies each transformer to the right columns and combines results for ML models

In [7]:
# Pipeline Format 
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features)
    ]
)

preprocessor just stores the data there , to put it in actual array formmat, we need fit.transfrom. note: if preprocessor was not the name, and something like shahrukh then x = shahrukh.fit_transform(x) wouldve worked the same

In [25]:
x = preprocessor.fit_transform(x)

In [30]:
x

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]], shape=(1000, 19))

#### Seperate Data into Train and Test Data

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)
x_train.shape, x_test.shape

((800, 19), (200, 19))

Creating Ecvaluation Model to evaluate the predcition and true values ( after we are done with the model development)

###### Pipeline → during training/testing, after predictions are made (model.predict(x_test)), this function is called to evaluate performance before saving or deploying the model.

In [32]:
def evaluate_model(true, predicted):
    mse = mean_squared_error(true, predicted)
    mae = mean_absolute_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)

    return mse, mae, rmse, r2_square
    


In [33]:
# Models

models = {
    "LinearRegression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Neighbors Regressor" : KNeighborsRegressor(),
    "Decision Tree Regressor" : DecisionTreeRegressor(),
    "RandomForestRegressor" : RandomForestRegressor(),
    "XGBRegressor" : XGBRegressor(),
    "CatBoostingRegressor" : CatBoostRegressor(verbose=False),
    "AdaBoostRegressor" : AdaBoostRegressor()    
}

In [44]:
## visualize the model performance function. ( llm help)
def print_model_performance(model_name, train_metrics, test_metrics):
    train_mse, train_mae, train_rmse, train_r2 = train_metrics
    test_mse, test_mae, test_rmse, test_r2 = test_metrics

    print(f"{'='*50}")
    print(f"{'MODEL: ' + model_name:^50}")  # Centered model name
    print(f"{'-'*50}")

    # Training Performance
    print(f"{'Training Performance':^50}")
    print(f"{'MSE':25}: {train_mse:.4f}")
    print(f"{'MAE':25}: {train_mae:.4f}")
    print(f"{'RMSE':25}: {train_rmse:.4f}")
    print(f"{'R^2 Score':25}: {train_r2:.4f}")
    print(f"{'-'*50}")

    # Test Performance
    print(f"{'Test Performance':^50}")
    print(f"{'MSE':25}: {test_mse:.4f}")
    print(f"{'MAE':25}: {test_mae:.4f}")
    print(f"{'RMSE':25}: {test_rmse:.4f}")
    print(f"{'R^2 Score':25}: {test_r2:.4f}")
    print(f"{'='*50}\n")


In [48]:
train_metrics = evaluate_model(y_train, y_train_pred)
test_metrics = evaluate_model(y_test, y_test_pred)
## # returns: (mse, mae, rmse, r2)

print_model_performance(model_name=list(models.keys())[i],
                        train_metrics=train_metrics,
                        test_metrics=test_metrics)

r2_list.append(test_metrics[3])  # R^2 score for test set


             MODEL: LinearRegression              
--------------------------------------------------
               Training Performance               
MSE                      : 28.3349
MAE                      : 4.2667
RMSE                     : 5.3231
R^2 Score                : 0.8743
--------------------------------------------------
                 Test Performance                 
MSE                      : 29.0952
MAE                      : 4.2148
RMSE                     : 5.3940
R^2 Score                : 0.8804



In [18]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
